## Lectura de librerias

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import snowflake.connector
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
import config 

Se lee la base previamente limpiada en el proceso anterior

In [2]:
curated = pd.read_csv('D:\Proyecto 2\Data\Clean\curated.csv')
curated.head()

,transaction_id,user_id,transaction_amount,transaction_type,time_of_transaction,device_used,location,previous_fraudulent_transactions,account_age,number_of_transactions_last_24h,payment_method,fraudulent
0,T1,4174,1292.76,ATM Withdrawal,16.00000,Tablet,San Francisco,0,119,13,Debit Card,0
1,T2,4507,1554.58,ATM Withdrawal,13.00000,Mobile,New York,4,79,3,Credit Card,0
2,T3,1860,2395.02,ATM Withdrawal,11.48752,Mobile,Unknown,3,115,9,Unknown,0
3,T4,2294,100.10,Bill Payment,15.00000,Desktop,Chicago,4,3,4,UPI,0
4,T5,2130,1490.50,POS Payment,19.00000,Mobile,San Francisco,2,57,7,Credit Card,0


In [3]:
curated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51000 entries, 0 to 50999
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   transaction_id                    51000 non-null  object 
 1   user_id                           51000 non-null  int64  
 2   transaction_amount                51000 non-null  float64
 3   transaction_type                  51000 non-null  object 
 4   time_of_transaction               51000 non-null  float64
 5   device_used                       51000 non-null  object 
 6   location                          51000 non-null  object 
 7   previous_fraudulent_transactions  51000 non-null  int64  
 8   account_age                       51000 non-null  int64  
 9   number_of_transactions_last_24h   51000 non-null  int64  
 10  payment_method                    51000 non-null  object 
 11  fraudulent                        51000 non-null  int64  
dtypes: f

In [4]:
print("Valores unicos para Transaction_ID:", len(curated['transaction_id'].unique()))
print("Valores unicos para User_ID:", len(curated['user_id'].unique()))

Valores unicos para Transaction_ID: 50000
Valores unicos para User_ID: 4000


Se crea una copia de la base para no modificarla directamente

In [5]:
clean = curated.copy()

In [ ]:
# Se decide eliminar estas columna porque no aporta valor al analisis
clean.drop(columns=['transaction_id'], inplace=True)

In [ ]:
# Se crea esta nueva variable para saber cuantas transacciones fraudulentas ha tenido el usuario en el pasado y cada cuanto podría volver a tener una
clean['fraud_per_month'] = np.where(
    clean['previous_fraudulent_transactions'] == 0,
    0,
    clean['account_age'] / clean['previous_fraudulent_transactions']
).round(0).astype(int)

In [8]:
clean.head()

,user_id,transaction_amount,transaction_type,time_of_transaction,device_used,location,previous_fraudulent_transactions,account_age,number_of_transactions_last_24h,payment_method,fraudulent,fraud_per_month
0,4174,1292.76,ATM Withdrawal,16.00000,Tablet,San Francisco,0,119,13,Debit Card,0,0
1,4507,1554.58,ATM Withdrawal,13.00000,Mobile,New York,4,79,3,Credit Card,0,20
2,1860,2395.02,ATM Withdrawal,11.48752,Mobile,Unknown,3,115,9,Unknown,0,38
3,2294,100.10,Bill Payment,15.00000,Desktop,Chicago,4,3,4,UPI,0,1
4,2130,1490.50,POS Payment,19.00000,Mobile,San Francisco,2,57,7,Credit Card,0,28


In [ ]:
#Se crea una copia final del dataset para el modelo de machine learning usando la tecnica de one hot encoding para las variables categoricas y de esta manera poder usar el dataset en el modelado final
ml_dataset = clean.copy()

categorical_features = ['transaction_type', 'device_used', 'location', 'payment_method']

df_train = pd.get_dummies(ml_dataset, columns=categorical_features)

In [10]:
df_train.head()

,user_id,transaction_amount,time_of_transaction,previous_fraudulent_transactions,account_age,number_of_transactions_last_24h,fraudulent,fraud_per_month,transaction_type_ATM Withdrawal,transaction_type_Bank Transfer,...,location_New York,location_San Francisco,location_Seattle,location_Unknown,payment_method_Credit Card,payment_method_Debit Card,payment_method_Invalid Method,payment_method_Net Banking,payment_method_UPI,payment_method_Unknown
0,4174,1292.76,16.00000,0,119,13,0,0,1,0,...,0,1,0,0,0,1,0,0,0,0
1,4507,1554.58,13.00000,4,79,3,0,20,1,0,...,1,0,0,0,1,0,0,0,0,0
2,1860,2395.02,11.48752,3,115,9,0,38,1,0,...,0,0,0,1,0,0,0,0,0,1
3,2294,100.10,15.00000,4,3,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2130,1490.50,19.00000,2,57,7,0,28,0,0,...,0,1,0,0,1,0,0,0,0,0


Se exporta la base final a la carpeta donde se encontrará el dataset a usar para el modelado

In [11]:
df_train.to_csv('D:\Proyecto 2\Data\Processed\ml_dataset.csv', index=False)